In [1]:
import ISRAnalysis as an

Welcome to JupyROOT 6.14/04


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import sys

# Unfolding for detector (up to dressed lepton level) 

In [3]:
binDef = "FineCoarse"
channel = "electron"
year = "2016"
doSys = True
regularisation_mode = 0
doInputStat = True
doRMStat = True

# Redirect STDOUT and STDERR
sys.stderr = open("./log/stderr_"+channel+"_"+year+".txt", "w")
sys.stdout = open("./log/stdout_"+channel+"_"+year+".txt", "w")

detUnfold = an.ISRAnalysis("DetUNFOLD", year, channel, regularisation_mode, doInputStat, doRMStat, "matrix", 
                           "Detector_Dressed_DRp1_Fiducial", binDef, "")

In [4]:
# Nominal
detUnfold.setInputHist(useMCInput=False, unfoldObj=None, dirName="Detector")

In [5]:
detUnfold.setUnfoldBkgs(dirName="Detector")

In [6]:
detUnfold.subFake() # Subract DY fake (out of fiducial volume at gen level)

In [7]:
if doSys == True:
    if channel == "muon" :
        systematics = {
                       "ID": ["IdSFUp", "IdSFDown"], 
                       "ISO":["IsoSFUp", "IsoSFDown"], 
                       "TRG":["trgSFUp", "trgSFDown"], 
                       "L1Prefie":["L1PrefireUp","L1PrefireDown"], 
                       "PU": ["PUUp", "PUDown"],
                       "Bveto":     ["bvetoUp", "bvetoDown"],
                       #"#ZptCorr": ["ZpTCorrected", "Nominal"], # use different matrix file
                       #"LepScale": ["LepScaleUp", "LepScaleDown"],
                       "Unfolding": ["unfoldIterEM", "unfoldNominal"],
                       "Scale":["ScaleABUp", "ScaleABDown", "ScaleAUp", "ScaleADown", "ScaleBUp", "ScaleBDown"], 
                       "AlphaS":["AlphaSUp", "AlphaSDown"],
                       "PDF":["PDFerror{:0>3}".format(str(i)) for i in range(1,101)],
                       #"Background": ["NormUp", "NormDown"],
                       }
    
    if channel == "electron" :
        
            # 1. Input distribution 2. Background distribution 3. Response matrix
            # 
            # Type 1: Change input(data) and background histogram and response matrix (ex. Lepton scale)
            # Type 2: Change background histograms and response matrix (ex. Lepton ID SF)
            # Type 3: Change only background histotrams (Background normalization)
            # Type 4: Only response matrix changes (ex. Unfolding model systematic or statistics)
            # Type 5: Only unfolding setting changes (for unfolding mode, etc)
            #
            # Lets make a dictionary for all the systematics, and then select for the specific data year
            
            
            systematics = {
            "Type_2": ["IdSFUp", "IdSFDown", "recoSFUp","recoSFDown", "trgSFUp", "trgSFDown", "PUUp", "PUDown", "bvetoUp", "bvetoDown", 
                       "ScaleABUp", "ScaleABDown", "ScaleAUp", "ScaleADown", "ScaleBUp", "ScaleBDown"],
            "Type_4": ["UnfoldingModel"]
            }
            ''' 
            systematics = {
                       "ID":        ["IdSFUp", "IdSFDown"],
                       "Reco":      ["recoSFUp","recoSFDown"], 
                       "TRG":       ["trgSFUp", "trgSFDown"], 
                       #"TRGDZ":       ["trgSFDZUp", "trgSFDZDown"],
                       "L1Prefie":  ["L1PrefireUp","L1PrefireDown"], 
                       "PU":        ["PUUp", "PUDown"],
                       "Bveto":     ["bvetoUp", "bvetoDown"],
                       #"Background": ["NormUp", "NormDown"],
                       #"ZptCorr": ["ZpTCorrected", "Nominal"],
                       #"LepScale": ["LepScaleUp", "LepScaleDown"],
                       #"LepRes": ["LepResUp", "LepResDown"],
                       #"Unfolding":  ["unfoldIterEM"],
                       "Scale":      ["ScaleABUp", "ScaleABDown", "ScaleAUp", "ScaleADown", "ScaleBUp", "ScaleBDown"], 
                       "AlphaS":     ["AlphaSUp", "AlphaSDown"],
                       #"PDF":        ["PDFerror{:0>3}".format(str(i)) for i in range(1,101)],
        }
        '''
    
    for sysType, sysNames in systematics.items():
        for sysName in sysNames:
            
            detUnfold.setSystematics(sysType, sysName) # Currently, response matrix set here
            
            detUnfold.setInputHist(False, None, "Detector", sysType, sysName)
            detUnfold.setUnfoldBkgs("Detector", sysType, sysName) 
            detUnfold.subFake("Detector_DY_Fake", sysType, sysName)

In [8]:
detUnfold.checkMatrixCond()

In [9]:
detUnfold.doUnfold() # nominal unfold

In [10]:
detUnfold.doAcceptance(isFSR=False, outName="DetEfficienyCorr")
detUnfold.closeOutFiles()
detUnfold.combineOutFiles()
# Combine pt and mass output root files

# Unfolding for QED FSR

In [ ]:
binDef = "CoarseCoarse"
phase_space = "Fiducial"

fsrUnfold = an.ISRAnalysis("FSRUNFOLD", year, channel, 0, True, "fsr_matrix", 
                                     "Dressed_DRp1_Dressed_DR4PI_"+phase_space, "Dressed_DRp1_Dressed_DR4PI", binDef, "")

In [ ]:
fsrUnfold.setInputHist(isSys=False, unfoldObj=detUnfold) # Set nominal input using previous unfolding result
fsrUnfold.subFake(isSys=False, dirName="Dressed_DRp1_DY_Fake", sysName="Nominal", isFSR=True)

In [ ]:
#fsrUnfold.drawResponseM("Pt")
#fsrUnfold.drawResponseM("Mass")

fsrUnfold.checkMatrixCond()

In [ ]:
if doSys :
    fsr_systematics = {
                       "Scale":["ScaleABUp", "ScaleABDown", "ScaleAUp", "ScaleADown", "ScaleBUp", "ScaleBDown"], 
                       "AlphaS":["AlphaSUp", "AlphaSDown"],
                       #"PDF":["PDFerror{:0>3}".format(str(i)) for i in range(1,101)],
                       "Unfolding": ["unfoldIterEM"],
                       #"FSR": ["fsrPHOTOS","fsrPYTHIA"],
    }
    
    for sysName, postfixs in fsr_systematics.items():
        for postfix in postfixs:
            fsrUnfold.setSystematics(sysName, postfix, True) # Create TUnfoldDensity objects for systematics
            
            if "FSR" in sysName:
                fsrUnfold.setInputHist(False, detUnfold, "", True, sysName, postfix)
                fsrUnfold.subFake(True, "Dressed_DRp1_DY_Fake", sysName, postfix, postfix, True)

In [ ]:
fsrUnfold.setFromPreviousUnfold(detUnfold) # Set using previous unfolding result for systematics

In [ ]:
if doSys:
    for sysName, postfixs in systematics.items():
        for postfix in postfixs:
            fsrUnfold.subFake(True, "Dressed_DRp1_DY_Fake", sysName, postfix, "", True)

In [ ]:
fsrUnfold.doUnfold() # do nominal unfold

In [ ]:
#fsrUnfold.doStatUnfold() # NOTE: doStatUnfold() must be done after setMeanValues() and before any other processes

# Acceptance correction 

In [ ]:
fsrUnfold.doAcceptance(isFSR=True)

In [ ]:
fsrUnfold.closeOutFiles()
fsrUnfold.combineOutFiles()